In [1]:
import cv2
import numpy as np
import time
from deep_sort_realtime.deepsort_tracker import DeepSort
from ultralytics import YOLO
import torch
import threading
import queue

class YoloDetector:
    def __init__(self):
        self.model = YOLO('yolov8n.pt')
        self.classes = self.model.names
        self.device = 'cuda' if torch.cuda.is_available() else 'cpu'
        print('Using Device: ', self.device)

    def score_frame(self, frame):
        results = self.model(frame, device=self.device)
        boxes = results[0].boxes
        labels = boxes.cls
        confidences = boxes.conf
        cord = boxes.xyxy
        return labels, cord, confidences

    def plot_boxes(self, results, frame, confidence=0.65):
        labels, cord, confidences = results
        detections = []

        for label, conf, xyxy in zip(labels, confidences, cord):
            if conf >= confidence and self.classes[int(label)] == 'person':
                x1, y1, x2, y2 = map(int, xyxy.cpu().numpy())
                detections.append(([x1, y1, int(x2 - x1), int(y2 - y1)], conf.item(), 'person'))

        return frame, detections

class MultiCameraTracker:
    def __init__(self):
        self.object_tracker1 = DeepSort(max_age=5000, n_init=2, nn_budget=None, 
                                        embedder="mobilenet", embedder_gpu=True)
        self.object_tracker2 = DeepSort(max_age=5000, n_init=2, nn_budget=None, 
                                        embedder="mobilenet", embedder_gpu=True)
        self.embeddings_dict = {}  # Shared embeddings dictionary across both cameras
        self.similarity_threshold = 0.75

    def update_tracks(self, detections1, detections2, frame1, frame2):
        tracks1 = self.object_tracker1.update_tracks(detections1, frame=frame1)
        tracks2 = self.object_tracker2.update_tracks(detections2, frame=frame2)

        self.match_across_cameras(tracks1, tracks2)

        return tracks1, tracks2

    def match_across_cameras(self, tracks1, tracks2):
        all_tracks = tracks1 + tracks2  # Combine tracks from both cameras

        for track in all_tracks:
            if not track.is_confirmed():
                continue
            embedding = np.array(track.features).flatten()  # Ensure it's a 1D array
            track_id = track.track_id

            best_match = None
            best_similarity = self.similarity_threshold

            # Search for the most similar existing embedding in `self.embeddings_dict`
            for stored_id, stored_embedding in self.embeddings_dict.items():
                stored_embedding = np.array(stored_embedding).flatten()  # Flatten for compatibility

                if embedding.shape != stored_embedding.shape:
                    continue  # Skip if shape mismatch

                similarity = np.dot(embedding, stored_embedding) / (np.linalg.norm(embedding) * np.linalg.norm(stored_embedding))

                if similarity > best_similarity:
                    best_similarity = similarity
                    best_match = stored_id

            # Assign or update track ID based on the match
            if best_match:
                # ReID success: assign the existing track ID
                self.embeddings_dict[track_id] = self.embeddings_dict[best_match]
                track.track_id = best_match  # Update track ID for re-identification
            else:
                # No match found: treat as new ID and add to embeddings dictionary
                self.embeddings_dict[track_id] = embedding

def process_camera(cap, detector, frame_queue, stop_event):
    frame_count = 0
    while not stop_event.is_set():
        success, img = cap.read()
        if not success:
            print(f"Failed to read frame from camera {cap.get(cv2.CAP_PROP_POS_FRAMES)}")
            break
        
        frame_count += 1
        if frame_count % 3 != 0:  # Process every 3rd frame
            continue
        
        results = detector.score_frame(img)
        img, detections = detector.plot_boxes(results, img)
        
        try:
            frame_queue.put((img, detections), block=False)
        except queue.Full:
            frame_queue.get()  # Remove oldest item if queue is full
            frame_queue.put((img, detections), block=False)

def main():
    detector = YoloDetector()
    multi_tracker = MultiCameraTracker()

    cap1 = cv2.VideoCapture(0)
    cap2 = cv2.VideoCapture(1)

    if not cap1.isOpened() or not cap2.isOpened():
        print("Error: Could not open cameras.")
        return

    cap1.set(cv2.CAP_PROP_FRAME_WIDTH, 640)  # Set resolution
    cap1.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)
    cap2.set(cv2.CAP_PROP_FRAME_WIDTH, 640)
    cap2.set(cv2.CAP_PROP_FRAME_HEIGHT, 480)

    frame_queue1 = queue.Queue(maxsize=100)
    frame_queue2 = queue.Queue(maxsize=100)

    stop_event = threading.Event()

    t1 = threading.Thread(target=process_camera, args=(cap1, detector, frame_queue1, stop_event))
    t2 = threading.Thread(target=process_camera, args=(cap2, detector, frame_queue2, stop_event))
    t1.start()
    t2.start()

    try:
        while cap1.isOpened() and cap2.isOpened():
            try:
                img1, detections1 = frame_queue1.get(timeout=1)
                img2, detections2 = frame_queue2.get(timeout=1)
            except queue.Empty:
                continue

            start = time.perf_counter()

            tracks1, tracks2 = multi_tracker.update_tracks(detections1, detections2, img1, img2)

            for tracks, img in [(tracks1, img1), (tracks2, img2)]:
                for track in tracks:
                    if not track.is_confirmed():
                        continue
                    track_id = track.track_id
                    ltrb = track.to_ltrb()
                    cv2.rectangle(img, (int(ltrb[0]), int(ltrb[1])), (int(ltrb[2]), int(ltrb[3])), (0, 0, 255), 2)
                    cv2.putText(img, f"ID: {track_id}", (int(ltrb[0]), int(ltrb[1]) - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.5, (0, 255, 0), 2)

            end = time.perf_counter()
            totalTime = end - start
            fps = 1 / totalTime

            cv2.putText(img1, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)
            cv2.putText(img2, f'FPS: {int(fps)}', (20, 70), cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 2)

            cv2.imshow('Camera 1', img1)
            cv2.imshow('Camera 2', img2)

            if cv2.waitKey(1) & 0xFF == 27:
                break

    except KeyboardInterrupt:
        print("Interrupted by user. Shutting down...")
    finally:
        stop_event.set()
        t1.join()
        t2.join()
        cap1.release()
        cap2.release()
        cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


100%|█████████████████████████████████████████████████████████████████████████████| 6.25M/6.25M [00:05<00:00, 1.19MB/s]


Using Device:  cuda

Ultralytics 8.3.8  Python-3.10.15 torch-2.4.1 CUDA:0 (Quadro M1200, 4096MiB)

0: 480x640 1 person, 46.0ms
Speed: 6.0ms preprocess, 46.0ms inference, 249.4ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 24.0ms
Speed: 4.0ms preprocess, 24.0ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 55.7ms
Speed: 7.0ms preprocess, 55.7ms inference, 6.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 26.2ms
Speed: 4.0ms preprocess, 26.2ms inference, 4.0ms postprocess per image at shape (1, 3, 480, 640)
YOLOv8n summary (fused): 168 layers, 3,151,904 parameters, 0 gradients, 8.7 GFLOPs

0: 480x640 (no detections), 29.0ms
Speed: 3.0ms preprocess, 29.0ms inference, 2.0ms postprocess per image at shape (1, 3, 480, 640)
0: 480x640 1 person, 31.2ms
Speed: 6.0ms preprocess, 31.2ms inference, 12.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 24.0ms
Speed: 3.0ms preprocess, 

KeyError: np.str_('1')

<div style="font-size: 24px; font-weight: bold; color: #ff0000; text-align: center; padding: 10px;">
    Error from Above Cell
</div>


In [ ]:
# ---------------------------------------------------------------------------
# KeyError                                  Traceback (most recent call last)
# Cell In[1], line 179
#     176         cv2.destroyAllWindows()
#     178 if __name__ == "__main__":
# --> 179     main()

# Cell In[1], line 144, in main()
#     140     continue
#     142 start = time.perf_counter()
# --> 144 tracks1, tracks2 = multi_tracker.update_tracks(detections1, detections2, img1, img2)
#     146 for tracks, img in [(tracks1, img1), (tracks2, img2)]:
#     147     for track in tracks:

# Cell In[1], line 47, in MultiCameraTracker.update_tracks(self, detections1, detections2, frame1, frame2)
#      45 def update_tracks(self, detections1, detections2, frame1, frame2):
#      46     tracks1 = self.object_tracker1.update_tracks(detections1, frame=frame1)
# ---> 47     tracks2 = self.object_tracker2.update_tracks(detections2, frame=frame2)
#      49     self.match_across_cameras(tracks1, tracks2)
#      51     return tracks1, tracks2

# File ~\anaconda3\envs\pytorch_env\lib\site-packages\deep_sort_realtime\deepsort_tracker.py:228, in DeepSort.update_tracks(self, raw_detections, embeds, frame, today, others, instance_masks)
#     226 # Update tracker.
#     227 self.tracker.predict()
# --> 228 self.tracker.update(detections, today=today)
#     230 return self.tracker.tracks

# File ~\anaconda3\envs\pytorch_env\lib\site-packages\deep_sort_realtime\deep_sort\tracker.py:97, in Tracker.update(self, detections, today)
#      94         self._next_id = 1
#      96 # Run matching cascade.
# ---> 97 matches, unmatched_tracks, unmatched_detections = self._match(detections)
#      99 # Update track set.
#     100 for track_idx, detection_idx in matches:

# File ~\anaconda3\envs\pytorch_env\lib\site-packages\deep_sort_realtime\deep_sort\tracker.py:151, in Tracker._match(self, detections)
#     142 unconfirmed_tracks = [
#     143     i for i, t in enumerate(self.tracks) if not t.is_confirmed()
#     144 ]
#     146 # Associate confirmed tracks using appearance features.
#     147 (
#     148     matches_a,
#     149     unmatched_tracks_a,
#     150     unmatched_detections,
# --> 151 ) = linear_assignment.matching_cascade(
#     152     gated_metric,
#     153     self.metric.matching_threshold,
#     154     self.max_age,
#     155     self.tracks,
#     156     detections,
#     157     confirmed_tracks,
#     158 )
#     160 # Associate remaining tracks together with unconfirmed tracks using IOU.
#     161 iou_track_candidates = unconfirmed_tracks + [
#     162     k for k in unmatched_tracks_a if self.tracks[k].time_since_update == 1
#     163 ]

# File ~\anaconda3\envs\pytorch_env\lib\site-packages\deep_sort_realtime\deep_sort\linear_assignment.py:147, in matching_cascade(distance_metric, max_distance, cascade_depth, tracks, detections, track_indices, detection_indices)
#     144     if len(track_indices_l) == 0:  # Nothing to match at this level
#     145         continue
# --> 147     matches_l, _, unmatched_detections = min_cost_matching(
#     148         distance_metric,
#     149         max_distance,
#     150         tracks,
#     151         detections,
#     152         track_indices_l,
#     153         unmatched_detections,
#     154     )
#     155     matches += matches_l
#     156 unmatched_tracks = list(set(track_indices) - set(k for k, _ in matches))

# File ~\anaconda3\envs\pytorch_env\lib\site-packages\deep_sort_realtime\deep_sort\linear_assignment.py:62, in min_cost_matching(distance_metric, max_distance, tracks, detections, track_indices, detection_indices)
#      59 if len(detection_indices) == 0 or len(track_indices) == 0:
#      60     return [], track_indices, detection_indices  # Nothing to match.
# ---> 62 cost_matrix = distance_metric(tracks, detections, track_indices, detection_indices)
#      63 cost_matrix[cost_matrix > max_distance] = max_distance + 1e-5
#      64 # indices = linear_assignment(cost_matrix)

# File ~\anaconda3\envs\pytorch_env\lib\site-packages\deep_sort_realtime\deep_sort\tracker.py:133, in Tracker._match.<locals>.gated_metric(tracks, dets, track_indices, detection_indices)
#     131 features = np.array([dets[i].feature for i in detection_indices])
#     132 targets = np.array([tracks[i].track_id for i in track_indices])
# --> 133 cost_matrix = self.metric.distance(features, targets)
#     134 cost_matrix = linear_assignment.gate_cost_matrix(
#     135     self.kf, cost_matrix, tracks, dets, track_indices, detection_indices, only_position=self.gating_only_position
#     136 )
#     138 return cost_matrix

# File ~\anaconda3\envs\pytorch_env\lib\site-packages\deep_sort_realtime\deep_sort\nn_matching.py:174, in NearestNeighborDistanceMetric.distance(self, features, targets)
#     172 cost_matrix = np.zeros((len(targets), len(features)))
#     173 for i, target in enumerate(targets):
# --> 174     cost_matrix[i, :] = self._metric(self.samples[target], features)
#     175 return cost_matrix

# KeyError: np.str_('1')